# Tutorial: Working with lists (Stateful `evaluate(env)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/list.html

This is the stateful variant: definitions are kept in a shared environment across cells.


In [1]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import EvaluationEnvironment, evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")

env = EvaluationEnvironment()
stateful = evaluate(env)


Using bqn_jax from: /home/johtok/repos/jax/xlapl/bqn-jax/src/bqn_jax/__init__.py


## Before You Start

1. Run the setup cell first to create `env` and `stateful`.
2. Run cells in order from top to bottom, because later cells may depend on earlier definitions.
3. Each `stateful("...")` call updates and reuses the same environment.
4. If results look odd after reruns, restart the kernel and run all cells again.
5. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## Examples

Quick warm-up examples before the deeper sections.


In [2]:
stateful('⟨1, 2, 3⟩')


Array([1., 2., 3.], dtype=float32, weak_type=True)

In [3]:
stateful('⟨1, 2, 3⟩ + 1')


Array([2., 3., 4.], dtype=float32, weak_type=True)

## List notation

Learn the compact list and strand notations used throughout the tutorial.


In [4]:
stateful('0‿1‿2')
stateful('(0‿1)‿2')
stateful('0‿(1‿2)')


[Array(0., dtype=float32, weak_type=True),
 Array([1., 2.], dtype=float32, weak_type=True)]

## BQN types

Keep in mind that array values can be simple or nested; later examples rely on this distinction.


## Arithmetic on lists

These examples apply scalar-style arithmetic directly to list data.


In [5]:
stateful('÷ ⟨2,3,4⟩')
stateful('"APL" + 1')
stateful('"31415" - \'0\'')
stateful('4‿3‿2‿1 ⋆ 1‿2‿3‿4')


Array([4., 9., 8., 1.], dtype=float32, weak_type=True)

In [6]:
stateful('2 × ⟨0‿2 ⋄ 1‿3‿5⟩')
stateful('⟨ 10, 20‿30 ⟩ + ⟨ 1‿2, 3 ⟩')


Array([[11., 12.],
       [23., 33.]], dtype=float32, weak_type=True)

## Some list functions

Common list primitives for joining, reversing, mapping, and folding.


In [7]:
stateful('⋈ "elt"')
stateful('2 ⋈ 4')


Array([2., 4.], dtype=float32)

In [8]:
stateful('⟨1,2,3⟩ ∾ "abc"')
stateful('0 ∾ ⟨1,2,3⟩')
stateful('"plural" ∾ \'s\'')


Array([112, 108, 117, 114,  97, 108, 115], dtype=int32)

In [9]:
stateful('⌽ "drawer"')


Array([114, 101, 119,  97, 114, 100], dtype=int32)

In [10]:
stateful('2 ⌽ ⟨0,1,2,3,4⟩')
stateful('¯1 ⌽ "bcdea"')


Array([ 97,  98,  99, 100, 101], dtype=int32)

In [11]:
stateful('⌽¨ "abcd"‿"ABCDEF"‿"01"')


[Array([100,  99,  98,  97], dtype=int32),
 Array([70, 69, 68, 67, 66, 65], dtype=int32),
 Array([49, 48], dtype=int32)]

In [12]:
stateful('"abc" ⋈¨ "ABC"')
stateful('"string"‿"list"‿"array" ∾¨ \'s\'')


[Array([115, 116, 114, 105, 110, 103, 115], dtype=int32),
 Array([108, 105, 115, 116, 115], dtype=int32),
 Array([ 97, 114, 114,  97, 121, 115], dtype=int32)]

In [13]:
stateful('+´ 2‿3‿4')
stateful('×´ 2‿3‿4')


Array(24., dtype=float32)

In [14]:
stateful('-´ 1‿2‿3‿4‿5')
stateful('1-2-3-4-5')


Array(3., dtype=float32, weak_type=True)

In [15]:
stateful('∾´ ⟨ "con", "cat", "enat", "e" ⟩')


Array([ 99, 111, 110,  99,  97, 116, 101, 110,  97, 116, 101], dtype=int32)

In [16]:
stateful('∾ ⟨ "con", "cat", "enat", "e" ⟩')


Array([ 99, 111, 110,  99,  97, 116, 101, 110,  97, 116, 101], dtype=int32)

## Example: base decoding

A step-by-step worked example that builds binary decoding with array operations.


In [17]:
stateful('↕ 8')


Array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [18]:
stateful('8‿4‿2‿1 ⋈¨ 1‿0‿0‿1')


Array([[8., 1.],
       [4., 0.],
       [2., 0.],
       [1., 1.]], dtype=float32)

In [19]:
stateful('+´ 8‿4‿2‿1 × 1‿0‿0‿1')


Array(9., dtype=float32)

In [20]:
stateful('2 ⋆ ↕4')
stateful('⌽2⋆↕4')
stateful('(⌽2⋆↕4) × "1001"-\'0\'')
stateful('+´ (⌽2⋆↕4) × "1001"-\'0\'')


Array(9., dtype=float32)

In [21]:
stateful('\'0\' -˜ "01001110"‿"01100101"‿"01110010"‿"01100100"‿"00100001"')


Array([[0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1]], dtype=int32)

In [22]:
stateful('+´¨ \'0\' -˜ "01001110"‿"01100101"‿"01110010"‿"01100100"‿"00100001"')


Array([4, 4, 4, 3, 2], dtype=int32)

In [23]:
stateful('(⌽2⋆↕8) × \'0\' -˜ "01001110"‿"01100101"‿"01110010"‿"01100100"‿"00100001"')


TypeError: mul: arrays must have the same number of dimensions, got {1, 2}

In [ ]:
stateful('"ab" ∾¨ ⟨ "cd", "ut" ⟩')
stateful('"ab"⊸∾¨ ⟨ "cd", "ut" ⟩')


In [ ]:
stateful('+´¨ (⌽2⋆↕8)⊸×¨ \'0\' -˜ "01001110"‿"01100101"‿"01110010"‿"01100100"‿"00100001"')


In [ ]:
stateful('@ + +´¨ (⌽2⋆↕8)⊸×¨ \'0\' -˜ "01001110"‿"01100101"‿"01110010"‿"01100100"‿"00100001"')


In [ ]:
stateful('+´ (⌽2⋆↕4) × "1001"-\'0\'')
stateful('+⟜(+˜)´ ⌽ "1001"-\'0\'')


## Summary

If any result is unclear, rerun the section above step by step before moving on.
